In [26]:
import os
import sys
import cv2
# import time
import json
import carla
import torch
import random
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as transforms
# from PIL import Image  
from process_frame import process_frame
# from model.ImitationCNN import ImitationCNN
from model.ImitationResNet import ImitationResNet

In [2]:
sys.path.append(r'/home/kaustubh/CARLA_15/PythonAPI/carla')
from agents.navigation.basic_agent import BasicAgent
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "models/bc_model.pth"
IMG_RGB = os.path.join('DAgger/rgb', "DAgger_image")
IMG_SEG = os.path.join('DAgger/seg', "DAgger_image")
JSON_PATH = os.path.join('DAgger', 'DAgger_log')

dagger_log = []

os.makedirs(IMG_RGB, exist_ok=True)
os.makedirs(IMG_SEG, exist_ok=True)
os.makedirs(JSON_PATH, exist_ok=True)

In [34]:
# model = ImitationCNN().to(device)
model = ImitationResNet().to(device='cpu')
model.load_state_dict(torch.load(MODEL_PATH)['model_state_dict'])
model.eval()

/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ImitationResNet(
  (conv1): Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace

In [35]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
traffic_manager = client.get_trafficmanager()
blueprint = world.get_blueprint_library()

In [36]:
vehicle_bp = blueprint.find('vehicle.volkswagen.t2_2021')
camera_bp = blueprint.find('sensor.camera.rgb')

In [37]:
spawn_points = world.get_map().get_spawn_points()
image = None
collision_detected = None
camera_sensor = None
vehicle = None
seg_image = None
rgb_image = None
collision_sensor = None

In [38]:
def image_callback(data):
    global image
    array = np.frombuffer(data.raw_data, dtype=np.uint8)
    array = np.reshape(array, (data.height, data.width, 4))[:, :, :3]
    image = array

def handle_collision(event):
    global collision_detected
    collision_detected = True

In [39]:
def pre_process(rgb_image, seg_image, transform):
    
    if transform:
        rgb_tensor = transform(rgb_image)
    else:
        rgb_tensor = transforms.ToTensor()(rgb_image)
        
    seg_image = np.array(seg_image)
    
    lane_mask = np.all(seg_image == [0, 255, 0], axis=2).astype(np.uint8)
    obs_mask = np.all(seg_image == [255, 0, 0], axis=2).astype(np.uint8)
    
    seg_tensor = torch.tensor(np.stack([lane_mask, obs_mask], axis=0), dtype=torch.float32)
    
    if seg_tensor.shape[1:] != rgb_tensor.shape[1:]:
        seg_tensor = F.interpolate(
            seg_tensor.unsqueeze(0),
            size=rgb_tensor.shape[1:],
            mode='nearest'
        ).squeeze(0)
    
    input_tensor = torch.cat([rgb_tensor, seg_tensor], dim=0)
    
    return input_tensor

In [40]:
def init():
    global camera_sensor, vehicle, rgb_image, seg_image, collision_sensor
    for actor in world.get_actors().filter("vehicle.volkswagen.t2_2021"):
        print(actor)
        actor.destroy()
    for actor in world.get_actors().filter("*sensor*"):
        print(actor)
        actor.destroy()
    camera_bp.set_attribute('image_size_x', '640')
    camera_bp.set_attribute('image_size_y', '360')

    vehicle_length = 4.442184
    # vehicle_width = 1.774566
    # vehicle_height = 1.987206

    camera_location = carla.Location(
        x=vehicle_length / 2.9 , 
        y=0,
        z=2.2
    )

    camera_transform = carla.Transform(camera_location)

    vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
    camera_sensor = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
    camera_sensor.listen(lambda image: image_callback(image))
    
    rgb_image = image
    seg_image = process_frame(image)
    transform = None
    
    collision_bp = blueprint.find('sensor.other.collision')
    collision_sensor = world.spawn_actor(collision_bp, carla.Transform(), attach_to=vehicle)
    collision_sensor.listen(handle_collision)


In [41]:
init()

Actor(id=395, type=vehicle.volkswagen.t2_2021)
Actor(id=397, type=sensor.other.collision)
Actor(id=396, type=sensor.camera.rgb)


In [ ]:
# # Testing Behaviour clonning, sometimes it works!
with torch.no_grad():
    while True:
        if collision_detected:
            init()
            collision_detected = False
            
        input_tensor = pre_process(rgb_image, seg_image, transform=None).unsqueeze(0).to('cpu')
        output = model(input_tensor)
        # print(output)
        steer, throttle, brake = output[0].cpu().numpy().tolist()
        # steer = float(f"{steer:.4f}")
        # steer = max(min(steer, 1.0), -1.0)    
        # throttle = max(min(throttle, 1.0), 0.0)  
        brake = max(min(brake, 1.0), 0.0)
        # if steer > 1 or throttle > 1:
        # print(steer, throttle, brake)
        
        processed_image = process_frame(image)
    
        if cv2.waitKey(1) == ord('q'):
            quit = True
            break

        combined_image = np.hstack((processed_image, image))

        cv2.imshow("Camera", combined_image)
        cv2.waitKey(1)

        vehicle.apply_control(carla.VehicleControl(
            # throttle= 0 if brake == 1.0 else 0.4,
            throttle= 0.5,
            steer=steer,
            # brake= 0 if brake < 7.0 else 1,
            brake= 0,
            hand_brake=False))
    
    cv2.destroyAllWindows()


Actor(id=398, type=vehicle.volkswagen.t2_2021)
Actor(id=400, type=sensor.other.collision)
Actor(id=399, type=sensor.camera.rgb)


Actor(id=401, type=vehicle.volkswagen.t2_2021)
Actor(id=403, type=sensor.other.collision)
Actor(id=402, type=sensor.camera.rgb)


Actor(id=404, type=vehicle.volkswagen.t2_2021)
Actor(id=405, type=sensor.camera.rgb)
Actor(id=406, type=sensor.other.collision)


Actor(id=407, type=vehicle.volkswagen.t2_2021)
Actor(id=409, type=sensor.other.collision)
Actor(id=408, type=sensor.camera.rgb)


Actor(id=410, type=vehicle.volkswagen.t2_2021)
Actor(id=412, type=sensor.other.collision)
Actor(id=411, type=sensor.camera.rgb)


...

Actor(id=416, type=vehicle.volkswagen.t2_2021)
Actor(id=418, type=sensor.other.collision)
Actor(id=417, type=sensor.camera.rgb)


Actor(id=419, type=vehicle.volkswagen.t2_2021)
Actor(id=421, type=sensor.other.collision)
Actor(id=420, type=sensor.camera.rgb)


Actor(id=422, type=vehicle.volkswagen.t2_2021)
Actor(id=424, type=sensor.other.collision)
Actor(id=423, type=sensor.camera.rgb)


Actor(id=425, type=vehicle.volkswagen.t2_2021)
Actor(id=427, type=sensor.other.collision)
Actor(id=426, type=sensor.camera.rgb)


Actor(id=428, type=vehicle.volkswagen.t2_2021)
Actor(id=430, type=sensor.other.collision)
Actor(id=429, type=sensor.camera.rgb)


In [ ]:
agent = BasicAgent(vehicle)

In [ ]:
goal = random.choice(spawn_points)
destination = goal.location
agent.set_destination(destination)

In [ ]:
expert_control = agent.run_step()

In [ ]:
print(expert_control)

In [ ]:
print(spawn_points[0])
print(goal)
map_ = world.get_map()

In [ ]:
grp = GlobalRoutePlanner(map_, sampling_resolution=2.0)
route = grp.trace_route(spawn_points[0].location, goal.location)
# route_coords = [[wp.transform.location.x, wp.transform.location.y] for wp, _ in route]
for wp, _ in route:
    world.debug.draw_point(
    wp.transform.location + carla.Location(z=0.5),
    size=0.2,
    color=carla.Color(0, 255, 0),
    life_time=300.0
                        )

In [ ]:
print(expert_control)

In [ ]:
while True:
    
    if random.random() < 0.6:
        print("Expert Control")
        control = agent.run_step()
        vehicle.apply_control(control)
        
    else:
        print("Stra")
        control = vehicle.apply_control(
                        carla.VehicleControl(
                            throttle=0.5, 
                            steer=0, 
                            brake=0
                        )
                    )

In [ ]:
frame = {'count': 0}
def save_data(image):
    
    processed_image = process_frame(image)
    img_name = f"{frame['count']:05d}.png"
    cv2.imwrite(os.path.join(IMG_RGB, img_name), image)
    cv2.imwrite(os.path.join(IMG_SEG, img_name), processed_image)

    expert_control = vehicle.get_control()
    expert_throttle = expert_control.throttle
    expert_steer = expert_control.steer
    expert_brake = expert_control.brake
    
    input_tensor = pre_process(rgb_image, seg_image, transform)
    with torch.no_grad():
        output = model(input_tensor).squeeze().cpu().numpy()
    model_steer, model_throttle, model_brake = output
    
    dagger_log.append({
        'img': img_name,
        'expert': {
            'steer': float(expert_steer),
            'throttle': float(expert_throttle),
            'brake': float(expert_brake),
        },
        'model': {
            'steer': float(model_steer),
            'throttle': float(model_throttle),
            'brake': float(model_brake),
        }
    })

    frame['count'] += 1
    if frame['count'] >= 5000:
        camera_sensor.stop()
        with open(os.path.join(JSON_PATH, "dagger_log.json"), "w") as f:
            json.dump(dagger_log, f, indent=2)
        print("DAgger data collection complete.")


In [ ]:
traffic_manager.ignore_lights_percentage(vehicle, 100.0)
traffic_manager.ignore_signs_percentage(vehicle, 100.0)

vehicle.set_autopilot(False)

# for i in range(5000):
#     try:
#         save_data(image)
#         time.sleep(0.1)
#     except:
#         pass
